**<h1>Додатки до дипломів (оцінки)</h1>**

**<h3><i>Dependencies</i></h3>**

In [1]:
import pandas as pd
import numpy as np
import re

**<h3><i>Settings</i></h3>**

In [2]:
# turn off chained assignment warning
pd.options.mode.chained_assignment = None

<h3><b><i>Constants</i></b></h3>

<h3><font color="purple"><b>Увага! Код коректно збирає дані лише з тих рейтингів, які були попередньо перевірені на відсутність одруків та співпадіння з навчальним планом</b></font></h3>

In [3]:
# ІНФОРМАЦІЯ ПРО ДИСЦИПЛІНИ
ALL_RATING_FILES = [
    ["reiting_20 - 21 (prepared).xlsx", 2020, 2021],
    ["reiting_21 - 22 (prepared).xlsm", 2021, 2022],
    ["reiting_22 - 23 (prepared).xlsx", 2022, 2023],
    ["reiting_23 - 24 (prepared).xlsx", 2023, 2024]
]

SUBJECT_TRANSLATE = "subject_translate.xlsx"
GRADUATES_CODES = "graduates_codes.xlsx"

**<h2>1. Збір даних</h2>**

**<h3>1.1 Читання метаданих про дисципліни</h3>**

In [4]:
def collect_subject_metadata(df):
    # Detect the cut points
    mask = df == "Прізвище, ім'я студента"
    semester_data_start  = np.where(mask.any())[0]

    mask = df == "Сер.зважений бал за навчання"
    semester_data_end  = np.where(mask.any())[0]

    metadata_all = list()
    for i in range(0, len(semester_data_start)): # для 2-х семестрів

        if len(semester_data_start) != len(semester_data_end):
            continue

        semester = df.iloc[:, semester_data_start[i]:semester_data_end[i]]

        # Detect rows with subject metadata
        credits_row_index = semester[semester.eq("Кредити").any(axis=1)].index.tolist()[0]
        teachers_row_index = credits_row_index + 5

        # Drop extra rows
        metadata = semester.loc[credits_row_index:teachers_row_index].dropna(axis=1)
        #metadata = metadata.iloc[:, 1:]
        metadata = metadata[:-1]

        # Rename row indexes
        new_index_names = ["credits", "hours", "type", "subject", "teacher_name"]
        metadata = metadata.rename(index=dict(zip(metadata.index, new_index_names)))

        metadata_all.append(metadata)

    return metadata_all

**<h3>1.2 Читання даних про оцінки за дисципліни</h3>**

In [5]:
def collect_student_data(df):
    # Detect the cut points
    mask = df == "Прізвище, ім'я студента"
    semester_data_start  = np.where(mask.any())[0]

    mask = df == "Сер.зважений бал за навчання"
    semester_data_end  = np.where(mask.any())[0]

    student_data_all = list()
    for i in range(0, len(semester_data_start)): # для 2-х семестрів

        if len(semester_data_start) != len(semester_data_end):
            continue

        semester = df.iloc[:, semester_data_start[i]:semester_data_end[i]]

        # If some row is not about student it has NaN in "Номер заліковки"
        semester = semester.dropna(subset=[semester.columns[1]])

        # Індекс на основі першого рядка
        semester.columns = semester.iloc[0]

        semester = semester[1:]

        student_data = semester.melt(id_vars = semester.columns[0:4])

        student_data.columns = ["student_name", "code", "payment", "group", "subject", "grade"]

        student_data_all.append(student_data)

    return student_data_all

**<h3>1.3 Функція для трансформації оцінки в ECTS</h3>**

In [6]:
def score_to_ects(score):
    if 90 <= score <= 100:
        return "A"
    elif 82 <= score < 90:
        return "B"
    elif 75 <= score < 82:
        return "C"
    elif 64 <= score < 75:
        return "D"
    elif 60 <= score < 64:
        return "E"
    elif 35 <= score < 60:
        return "FX"
    elif score < 35:
        return "F"
    else:
        return None

**<h3>1.4 Функція для трансформації оцінки в національну шкалу</h3>**

In [7]:
def score_to_national_grade(ects, type):
    if type == "з":
      return "Зараховано/Passed"

    if ects == "A":
        return "Відмінно/Excellent"
    elif ects == "B" or ects == "C":
        return "Добре/Good"
    elif ects == "D" or ects == "E":
        return "Задовільно/Satisfactory"
    else:
        return None

**<h3>1.5 Скорочення ПІБ до ПІ</h3>**

In [8]:
def short_student_name(name):
  parts = name.split()
  return ' '.join(parts[:2]).strip()

**<h3>1.6 Розрахунок семестру</h3>**

In [9]:
def get_semester(group, semester):
  course = int(str(group)[0])
  return 2 * (course - 1) + (semester + 1)

<h3><b>1.7 Логіка для вибору оцінки та визначення суми годин та кредитів у випадку дублювання дисципліни</b></h3>

In [10]:
def get_priority(type):
  if type == "і":
    return 1
  elif type == "з":
    return 2
  elif type == "а":
    return 3

  return None

<h2><b>2. Читання даних</b></h2>

**<h3>2.1 Читання кожного аркуша (номер групи) з Excel-файла</h3>**

In [11]:
def xls_to_dataframe(file, study_year_start, study_year_end):
    groups = [sheet_name for sheet_name in file.sheet_names if re.match(r'^\d+м?$', sheet_name) ]

    data = list()
    for group in groups:
        df = file.parse(group)

        metadata_all = collect_subject_metadata(df)
        student_data_all = collect_student_data(df)

        for semester in range(0, len(metadata_all)):
            metadata = metadata_all[semester]
            student_data = student_data_all[semester]

            merged_dataset = pd.merge(student_data, metadata.transpose(), on='subject')
            merged_dataset["semester"] = semester

            data.append(merged_dataset)

    return pd.concat(data, axis=0)

**<h3>2.2 Читання та збереження даних з файлів</h3>**

In [12]:
# ДАНІ ПРО ОЦІНКИ
content = list()
for file_data in ALL_RATING_FILES:
  file = pd.ExcelFile(file_data[0])
  content.append(xls_to_dataframe(file, study_year_start=file_data[1], study_year_end=file_data[2]))

data = pd.concat(content, ignore_index=True, axis=0)

# Вважати що рядок містить дані про студента лише тоді коли вказано номер групи та спосіб фінансування
data = data[data["group"].notna() & data["payment"].notna()]

# Не враховувати пусті комірки (предмети з яких ще не було іспитів / заліків / курсових тощо)
data = data[data["grade"].notna() & (data["grade"] != "#ЗНАЧ!")]

subject_translate = pd.read_excel(SUBJECT_TRANSLATE)

**<h2>3. Обробка даних</h2>**

<h3><b>3.1 Змінити тип даних для оцінки</b></h3>

In [13]:
data["grade"] = data["grade"].replace(r".*н.*", 0, regex=True).astype(int)

<h3><b>3.2 Додати бал ECTS</b></h3>

In [14]:
data['grade_ects'] = data.apply(lambda row: score_to_ects(row['grade']), axis=1)

<h3><b>3.3 Додати бал за національною шкалою</b></h3>

In [15]:
data['national_grade'] = data.apply(lambda row: score_to_national_grade(row['grade_ects'], row['type']), axis=1)

<h3><b>3.4 ПІБ скоротити до ПІ</b></h3>

In [16]:
data['student_name'] = data.apply(lambda row: short_student_name(row['student_name']), axis=1)

<h3><b>3.5 Визначення номеру семестру (наскрізна нумерація)</b></h3>

In [17]:
data["semester"] = data.apply(lambda row: get_semester(row['group'], row['semester']), axis=1)

<h3><b>3.6 Форматування номеру групи</b></h3>

In [18]:
data["group"] = data["group"].astype(int).astype(str)

<h3><b>3.7 Встановлення пріоритету оцінки в залежності від типу підсумкового контролю</b></h3>

In [19]:
data["priority"] = data.apply(lambda row: get_priority(row['type']), axis=1)

<h3><b>3.8 Попередня фільтрація даних (залишити лише випускників)</b></h3>

In [20]:
graduates = set(pd.read_excel(GRADUATES_CODES)["student_code"])
data = data[data["code"].isin(graduates)]

<h3><b><font color="red">3.9 Важливо! Відсортувати перед групуванням для вибору оцінки згідно з пріоритетом іспиту, заліку та атестації</font></b></h3>

In [21]:
data = data.sort_values(by=["subject", "priority", "semester"], ascending=[True, True, False])

<h2><b>4. Формування та збереження таблиці оцінок</b></h2>

<h3><b>4.1 Агрегація</b></h3>

In [22]:
result = data.groupby(['code', 'subject']).agg({
    'student_name': 'first',
    'semester': 'first',
    'credits': 'sum',
    'hours': 'sum',
    'type': 'first',
    'grade_ects': 'first',
    'national_grade': 'first',
    'grade': 'first'
}).reset_index()

<h3><b>4.2 Розрахунок кількості годин на основі кількості кредитів (1 кредит = 30 годин)</b></h3>

In [23]:
result["hours"] = result["credits"] * 30

<h3><b>4.3 Форматований запис даних про кредити та години та назву дисципліни</b></h3>

In [24]:
result = pd.merge(result, subject_translate, on='subject', how='left').drop_duplicates().sort_values(by=["semester", "subject"])
result["credits_hours"] = result.apply(lambda row: f"{row['credits']:.1f} ({row['hours']:.0f})", axis=1)
result["subject"] = result.apply(lambda row: f"{row['subject']} / {row['subject_eng']}", axis=1)

<h3><b>4.4 Запис файлу</b></h3>

In [25]:
result.to_csv('marks.csv')

<h2><b>5. Виписка оцінок на одного студента (для перевірки)</b></h2>

In [26]:
code = 22010917

In [27]:
student_data = result[result['code'] == code]
student_data

,code,subject,student_name,semester,credits,hours,type,grade_ects,national_grade,grade,subject_eng,credits_hours
10325,22010917,Історія та культура України / History and cult...,Олексієнко Володимир,1,6,180,з,A,Зараховано/Passed,90,History and culture of Ukraine,6.0 (180)
10381,22010917,Основи програмної інженерії / Fundamentals of ...,Олексієнко Володимир,1,5,150,і,A,Відмінно/Excellent,100,Fundamentals of software engineering,5.0 (150)
10405,22010917,Українська мова (за професійним спрямуванням) ...,Олексієнко Володимир,1,3,90,і,A,Відмінно/Excellent,90,Ukrainian language (by professional direction),3.0 (90)
10415,22010917,Філософія / Philosophy,Олексієнко Володимир,1,3,90,з,A,Зараховано/Passed,95,Philosophy,3.0 (90)
10321,22010917,Іноземна мова (англійська) (додатково) (ДОП) /...,Олексієнко Володимир,2,5.5,165.0,а,A,Відмінно/Excellent,96,Foreign language (English) (additional) (DOP),5.5 (165)
10323,22010917,Інформаційні технології (ДОП) / Information te...,Олексієнко Володимир,2,8.0,240.0,з,A,Зараховано/Passed,100,Information technology (IT),8.0 (240)
10331,22010917,Архітектура комп'ютерів / Computer architecture,Олексієнко Володимир,2,4,120,з,A,Зараховано/Passed,100,Computer architecture,4.0 (120)
10339,22010917,Вища математика / Higher mathematics,Олексієнко Володимир,2,8.0,240.0,і,A,Відмінно/Excellent,91,Higher mathematics,8.0 (240)
10343,22010917,Дискретні структури та дискретна математика / ...,Олексієнко Володимир,2,7.0,210.0,і,A,Відмінно/Excellent,100,Discrete structures and discrete mathematics,7.0 (210)
10382,22010917,Основи програмування / Fundamentals of program...,Олексієнко Володимир,2,8,240,і,A,Відмінно/Excellent,100,Fundamentals of programming,8.0 (240)


<h3><b><i>P. S. Перевірка суми кредитів та годин (ДОПи не рахувати в загальну суму!!!)</b></i></h3>

<b>Кредити:</b>

In [28]:
student_data = student_data[~student_data['subject'].str.contains("ДОП")]
student_data["credits"].sum()

231.0

<b>Години:</b>

In [29]:
student_data["hours"].sum()

6930.0

<h3><b><font color="red">6. Здобувачі які не набрали необхідну кількість кредитів (поновлені, коледж, перейшли з заочки тощо):</font></b></h3>

In [30]:
exceptions = result[~result['subject'].str.contains("ДОП")]
exceptions = exceptions.groupby("code").agg({
    'student_name': 'first',
    'credits': 'sum'
}).reset_index()

exceptions[~exceptions["credits"].isin({108, 231})]

,code,student_name,credits
0,21610307,Косолап Ілля,121.5
1,21610902,Гудзь Дмитро,48.0
2,21617108,Мальцев Євген,48.0
3,21810209,Дмитрук Владислав,56.5
9,21910101,Артим Владислав,121.5
10,21910527,Фалько Дмитро,172.5
11,21910532,Шкромида Віталій,25.5
12,21910804,Біба Євгеній,25
15,22010106,Дзина Βлада,226.0
133,22017113,Шерстюк Олена,221.5



<h3><b><font color="red">7. Здобувачі які мали борги на момент завантаження та обробки локальної копії рейтингів (УВАЖНО ПЕРЕДИВИТИСЯ ДОДАТКИ):</font></b></h3>

Перелік боргів:

In [31]:
academic_debts = result[result["grade"] < 60].sort_values(by="student_name")
academic_debts = academic_debts[["student_name", "subject", "semester"]]
academic_debts.head()

,student_name,subject,semester
7104,Гοлубев Денис,Алгоритми та методи обчислень / Algorithms and...,5
7153,Гοлубев Денис,Передатестаційна практика / Pre-certification ...,8
54,Гудзь Дмитро,Дослідження операцій / Operations Research,11
57,Гудзь Дмитро,Контрольно-вимірювальні прилади та датчики сис...,11
60,Гудзь Дмитро,"Моделі, алгоритмізація і САК / Models, algorit...",12


Кількість боргів:

In [32]:
academic_debts.groupby("student_name").agg({
    'subject': 'count',
}).reset_index()

,student_name,subject
0,Гοлубев Денис,2
1,Гудзь Дмитро,6
2,Дзина Βлада,10
3,Жердецький Владислав,1
4,Кляцький Нікіта,6
5,Коваленко Мілана,1
6,Козій Олександр,7
7,Косолап Ілля,2
8,Лялюк Βοлοдимир,1
9,Мальцев Євген,1
